# DataRobot Prediction Scoring Python Examples
### The examples below show how to get predictions (aka scoring) and prediction explanations in various ways using python.  This includes using our datarobot python module, the requests module, the batch scoring command line tool, and requesting predictions from a deployed model on the prediction server (vs a model on the modelling server).

In [1]:
import pandas as pd
import numpy as np
import datarobot as dr
import os
import time
import sys
import requests
import pprint
from pandas.io.json import json_normalize
pd.options.display.max_colwidth = 200
pd.options.display.max_columns = 200

# Set up
### Create train and test (prediction) data sets.  We'll primarily use the predictions in this notebook, but this can be handy when training projects.

In [2]:
#
# Train test split a dataset
#

SOURCE_FILE = 'data/DR_Demo_10K_Lending_Club_Loans.csv'
TRAIN_FILE = 'data/DR_Demo_10K_Lending_Club_Loans_train.csv'
PRED_FILE = 'data/DR_Demo_10K_Lending_Club_Loans_pred.csv'

df_full = pd.read_csv(SOURCE_FILE, encoding = 'ISO-8859-1')
target = 'is_bad'

# Shuffle the rows
df_full = df_full.sample(frac=1, random_state=0)

# Split 90% for training, 10% for predictions
split = int(df_full.shape[0] * .1)
df = df_full[split:]
df_pred = df_full[:split].copy()

# Drop the target from the prediction dataset
prediction_data = df_pred.drop(target, axis=1)

print('Full data shape:      ',df_full.shape)
print('Training data shape:  ',df.shape)
print('Prediction data shape:',prediction_data.shape)

df.to_csv(TRAIN_FILE, index=False)
prediction_data.to_csv(PRED_FILE, index=False)

Full data shape:       (10000, 34)
Training data shape:   (9000, 34)
Prediction data shape: (1000, 33)


In [12]:
# 
# Connect to DataRobot  
#
# Replace the API token and username with yours, along with the 
# depolyment id and/or project and model ids you are using.  
#

USERNAME = os.environ['DATAROBOT_USERNAME']
API_TOKEN = os.environ['DATAROBOT_API_TOKEN']
ENDPOINT = 'https://app.datarobot.com/api/v2'

dr.Client(token=API_TOKEN, endpoint=ENDPOINT)  

# If there is an existing deployment, we'll use the deployment id to get the 
# model and project currently deployed to it.  If not, enter it manually.
DEPLOYMENT_ID = '5c19273c06eeed008a2ac7f9'

deployment = dr.Deployment.get(deployment_id=DEPLOYMENT_ID)
print('deployment.id:', deployment.id)
print('deployment.label:', deployment.label)

PROJECT_ID = deployment.model.get('project_id')
MODEL_ID = deployment.model.get('id')

project = dr.Project.get(project_id=PROJECT_ID)
model = dr.Model.get(project=PROJECT_ID, model_id=MODEL_ID)
# datasets = project.get_datasets()

print('project:', project.id)
print('model:', model.id)

deployment.id: 5c19273c06eeed008a2ac7f9
deployment.label: is_bad Predictions - notebook code example
project: 5dcf56a344849b1adf9583e0
model: 5dcf56f1bc200d1b7e63900d


# 1. Using the datarobot module API to request predictions from the app modeling server

In [4]:
# 
# New Scoring Predictions directly on project and model object 
#

print('Uploading prediction dataset')
dataset_from_path = project.upload_dataset(PRED_FILE)

print('Request predictions')
predict_job = model.request_predictions(dataset_from_path.id)

print('Waiting for prediction calculations')
predictions = predict_job.get_result_when_complete()

predictions.head()

Uploading prediction dataset
Request predictions
Waiting for prediction calculations


,positive_probability,prediction,prediction_threshold,row_id,class_0.0,class_1.0
0,0.252143,0.0,0.5,0,0.747857,0.252143
1,0.257976,0.0,0.5,1,0.742024,0.257976
2,0.064656,0.0,0.5,2,0.935344,0.064656
3,0.103950,0.0,0.5,3,0.896050,0.103950
4,0.198613,0.0,0.5,4,0.801387,0.198613


## ...and to request prediction explanations

In [5]:
# 
# Per https://datarobot-public-api-client.readthedocs-hosted.com/en/v2.13.1/api/prediction_explanations.html
# you may need to do this:
#
# "In order to create PredictionExplanations for a particular model and dataset, you must first:
# 
#   Compute feature impact for the model via 
#   dr.Model.get_feature_impact()
# 
#   Compute a PredictionExplanationsInitialization for the model via 
# Let's time this:
t1 = time.time()
print('Initializing prediction explanations')
pei = dr.PredictionExplanationsInitialization.create(project.id, model.id)
print(pei)
print('Waiting for initialization')
print(pei.get_result_when_complete())
# 
#   Compute predictions for the model and dataset via 
#   dr.Model.request_predictions(dataset_from_path.id)
#

print('Creating prediction explanations')
pe_job = dr.PredictionExplanations.create(project.id, model.id,  dataset_from_path.id)

print('Waiting for job to complete')
pe = pe_job.get_result_when_complete()

print('- Time: %0.3f' % (time.time()-t1))
df_pe = pe.get_all_as_dataframe()
df_pe.head()

Initializing prediction explanations
Job(predictionExplanationsInitialization, status=inprogress)
Waiting for initialization
PredictionExplanationsInitialization(project_id=5dcf56a344849b1adf9583e0, model_id=5dcf56f1bc200d1b7e63900d)
Creating prediction explanations
Waiting for job to complete
- Time: 81.013


,row_id,prediction,class_0_label,class_0_probability,class_1_label,class_1_probability,explanation_0_feature,explanation_0_feature_value,explanation_0_label,explanation_0_qualitative_strength,explanation_0_strength,explanation_1_feature,explanation_1_feature_value,explanation_1_label,explanation_1_qualitative_strength,explanation_1_strength,explanation_2_feature,explanation_2_feature_value,explanation_2_label,explanation_2_qualitative_strength,explanation_2_strength
0,0,0.0,0.0,0.747857,1.0,0.252143,int_rate,17.56,1.0,+++,0.234768,term,60 months,1.0,++,0.187849,grade,E,1.0,++,0.134023
1,1,0.0,0.0,0.742024,1.0,0.257976,term,60 months,1.0,++,0.341669,sub_grade,E4,1.0,++,0.323003,int_rate,19.29,1.0,++,0.217547
2,2,0.0,0.0,0.935344,1.0,0.064656,int_rate,7.49,1.0,---,-0.384449,title,Debt Consolidation Loan,1.0,---,-0.207765,grade,A,1.0,--,-0.198126
3,3,0.0,0.0,0.896050,1.0,0.103950,annual_inc,38000,1.0,+++,0.372695,int_rate,7.51,1.0,---,-0.269522,title,Credit Card Debt Consolidation,1.0,--,-0.180576
4,4,0.0,0.0,0.801387,1.0,0.198613,int_rate,16.77,1.0,+++,0.295286,annual_inc,37000,1.0,+++,0.276217,inq_last_6mths,1,1.0,++,0.078378


# 2. Using requests.post to a deployment to get predictions or prediction explanations from a deployment and tbe dedicated prediction server

In [20]:
#
# This is example code taken from the Deployement > Integrations tab, with added code for a pandas dataframe 
# converted to submit as type json.
#
# The requests.post url route is different depending on requesting predictions or prediction explanations 
#
# Replace the API token, username and deployment id  with yours.  
#

class DataRobotPredictionError(Exception):
    """Raised if there are issues getting predictions from DataRobot"""

def _raise_dataroboterror_for_status(response):
    """Raise DataRobotPredictionError if the request fails along with the response returned"""
    try:
        response.raise_for_status()
    except requests.exceptions.HTTPError:
        err_msg = '{code} Error: {msg}'.format(
            code=response.status_code, msg=response.text)
        raise DataRobotPredictionError(err_msg)

import requests
import sys

DATAROBOT_KEY = '544ec55f-61bf-f6ee-0caf-15c7f919a45d'  # Use yours, as denoted in the Integrations example code
API_URL = 'https://cfds-ccm-prod.orm.datarobot.com/predApi/v1.0/deployments/{deployment_id}/predictions'

# Make predictions on your data
# The URL has the following format:
#     https://cfds-ccm-prod.orm.datarobot.com/predApi/v1.0/deployments/<DEPLOYMENT_ID>/predictions
# See docs for details:
#     app.datarobot.com/docs/users-guide/deploy/api/new-prediction-api.html

# For data from a file:
# Note: The charset should match the contents of the file.
# headers = {'Content-Type': 'text/plain; charset=UTF-8', 
#            'Authorization': 'Bearer {}'.format(API_TOKEN),
#            'datarobot-key': DATAROBOT_KEY}
# data = open(sys.argv[1], 'rb').read()

# For data from a dataframe:
# generate JSON version of the dataframe to pass to API:
headers = {'Content-Type': 'application/json; charset=UTF-8', 
           'Authorization': 'Bearer {}'.format(API_TOKEN),
           'datarobot-key': DATAROBOT_KEY}
data = pd.read_csv(PRED_FILE)
data = data.to_json(orient='records')

url = API_URL.format(deployment_id=DEPLOYMENT_ID)

# Prediction Explanations:
# See the documentation for more information:
# https://app.datarobot.com/docs/users-guide/predictions/api/new-prediction-api.html#request-pred-explanations
# Should you wish to include Prediction Explanations or Prediction Warnings in the result,
# Change the parameters below accordingly, and remove the comment from the params field below:

params = {
    # If explanations are required, uncomment the line below
    'maxExplanations': 3,
    # 'thresholdHigh': 0.5,
    # 'thresholdLow': 0.15,
    # Uncomment this for Prediction Warnings, if enabled for your deployment.
    # 'predictionWarningEnabled': 'true',
}
# Make API request for predictions
predictions_response = requests.post(
    url,
    data=data,
    headers=headers,
    # Prediction Explanations:
    # Uncomment this to include explanations in your prediction
    params=params,
)

# params = {
#     'maxCodes': 3,
# #     'thresholdHigh': 0.5,
# #     'thresholdLow': 0.15,
#     'thresholdHigh': 0.0,
#     'thresholdLow': 0.0,
# }
# # Make API request for predictions
# predictions_response = requests.post(
#     url,
#     auth=(USERNAME, API_TOKEN),
#     data=data,
#     headers=headers,
#     params=params,
# )
_raise_dataroboterror_for_status(predictions_response)

# This might be quite big depending on how many rows you submit for prediction...
predictions_response.json()

{'data': [{'predictionValues': [{'value': 0.2521431884, 'label': 1.0},
    {'value': 0.7478568116, 'label': 0.0}],
   'predictionThreshold': 0.5,
   'prediction': 0.0,
   'rowId': 0,
   'predictionExplanations': [{'featureValue': 17.56,
     'strength': 0.2347682566,
     'feature': 'int_rate',
     'qualitativeStrength': '+++',
     'label': 1.0},
    {'featureValue': ' 60 months',
     'strength': 0.18784916,
     'feature': 'term',
     'qualitativeStrength': '++',
     'label': 1.0},
    {'featureValue': 'E',
     'strength': 0.1340228025,
     'feature': 'grade',
     'qualitativeStrength': '++',
     'label': 1.0}]},
  {'predictionValues': [{'value': 0.2579760086, 'label': 1.0},
    {'value': 0.7420239914, 'label': 0.0}],
   'predictionThreshold': 0.5,
   'prediction': 0.0,
   'rowId': 1,
   'predictionExplanations': [{'featureValue': ' 60 months',
     'strength': 0.3416690774,
     'feature': 'term',
     'qualitativeStrength': '++',
     'label': 1.0},
    {'featureValue': 'E4

In [22]:
# Format the prediction response dict nested values output, each in its own column, 
# as it is using the datarobot module (and almost the same as the batch tool [see below])
df_response = json_normalize(predictions_response.json()['data'])

# Flatten out prediction values
pes = df_response.iloc[0]['predictionValues']
pe_keys = sorted(pes[0].keys())
num_pe = len(pes)

pe_keys = sorted(pes[0].keys())
for i in range(num_pe):
    cur_pe = pes[i]
    label = 'err'
    for key in pe_keys:
        series = df_response.apply(lambda x: pd.Series([x['predictionValues'][i].get(key)], index=[key]), axis=1)
        if key == 'label':
            label = series[key][0]
        col_name = 'class_%s' % (label)
        df_response[col_name] = series

# Flatten out prediction explanations
pes = df_response.iloc[0]['predictionExplanations']
pe_keys = sorted(pes[0].keys())
num_pe = len(pes)

pe_keys = sorted(pes[0].keys())
for i in range(num_pe):
    cur_pe = pes[i]
    for key in pe_keys:
        col_name = 'explanation_%s_%s' % (i, key)
        df_response[col_name] = df_response.apply(lambda x: pd.Series([x['predictionExplanations'][i].get(key)], index=[key]), axis=1)

df_response.head()

,prediction,predictionExplanations,predictionThreshold,predictionValues,rowId,class_1.0,class_0.0,explanation_0_feature,explanation_0_featureValue,explanation_0_label,explanation_0_qualitativeStrength,explanation_0_strength,explanation_1_feature,explanation_1_featureValue,explanation_1_label,explanation_1_qualitativeStrength,explanation_1_strength,explanation_2_feature,explanation_2_featureValue,explanation_2_label,explanation_2_qualitativeStrength,explanation_2_strength
0,0.0,"[{'featureValue': 17.56, 'strength': 0.2347682566, 'feature': 'int_rate', 'qualitativeStrength': '+++', 'label': 1.0}, {'featureValue': ' 60 months', 'strength': 0.18784916, 'feature': 'term', 'qu...",0.5,"[{'value': 0.2521431884, 'label': 1.0}, {'value': 0.7478568116, 'label': 0.0}]",0,0.252143,0.747857,int_rate,17.56,1.0,+++,0.234768,term,60 months,1.0,++,0.187849,grade,E,1.0,++,0.134023
1,0.0,"[{'featureValue': ' 60 months', 'strength': 0.3416690774, 'feature': 'term', 'qualitativeStrength': '++', 'label': 1.0}, {'featureValue': 'E4', 'strength': 0.3230031775, 'feature': 'sub_grade', 'q...",0.5,"[{'value': 0.2579760086, 'label': 1.0}, {'value': 0.7420239914, 'label': 0.0}]",1,0.257976,0.742024,term,60 months,1.0,++,0.341669,sub_grade,E4,1.0,++,0.323003,int_rate,19.29,1.0,++,0.217547
2,0.0,"[{'featureValue': 7.49, 'strength': -0.3844490573, 'feature': 'int_rate', 'qualitativeStrength': '---', 'label': 1.0}, {'featureValue': 'Debt Consolidation Loan', 'strength': -0.207765149, 'featur...",0.5,"[{'value': 0.0646559332, 'label': 1.0}, {'value': 0.9353440668, 'label': 0.0}]",2,0.064656,0.935344,int_rate,7.49,1.0,---,-0.384449,title,Debt Consolidation Loan,1.0,---,-0.207765,grade,A,1.0,--,-0.198126
3,0.0,"[{'featureValue': 38000.0, 'strength': 0.3726952939, 'feature': 'annual_inc', 'qualitativeStrength': '+++', 'label': 1.0}, {'featureValue': 7.51, 'strength': -0.2695220159, 'feature': 'int_rate', ...",0.5,"[{'value': 0.1039504294, 'label': 1.0}, {'value': 0.8960495706, 'label': 0.0}]",3,0.103950,0.896050,annual_inc,38000,1.0,+++,0.372695,int_rate,7.51,1.0,---,-0.269522,title,Credit Card Debt Consolidation,1.0,--,-0.180576
4,0.0,"[{'featureValue': 16.77, 'strength': 0.2952863213, 'feature': 'int_rate', 'qualitativeStrength': '+++', 'label': 1.0}, {'featureValue': 37000.0, 'strength': 0.2762166789, 'feature': 'annual_inc', ...",0.5,"[{'value': 0.1986127644, 'label': 1.0}, {'value': 0.8013872356, 'label': 0.0}]",4,0.198613,0.801387,int_rate,16.77,1.0,+++,0.295286,annual_inc,37000,1.0,+++,0.276217,inq_last_6mths,1,1.0,++,0.078378


# 3. Using the batch script tool

### This installs the batch prediction help code from:
### https://github.com/datarobot/batch-scoring
### pip install -U datarobot_batch_scoring

For example:
- batch_scoring --host=https://mycorp.orm.datarobot.com/ --user="greg@mycorp.com" --out=pred.csv 5545eb20b4912911244d4835 5545eb71b4912911244d4847 /home/greg/Downloads/diabetes_test.csv
- batch_scoring_sse --host=https://mycorp.orm.datarobot.com/ --out=pred.csv 0ec5bcea7f0f45918fa88257bfe42c09 /home/greg/Downloads/diabetes_test.csv
- batch_scoring_deployment_aware --host=https://mycorp.orm.datarobot.com/ --user="greg@mycorp.com" --out=pred.csv 5545eb71b4912911244d4848 /home/greg/Downloads/diabetes_test.csv


In [8]:
#
# Execute the batch_scoring script via a bash shell conmand
#

DATAROBOT_KEY = '544ec55f-61bf-f6ee-0caf-15c7f919a45d'  # used for deployments on the cloud app, not on-prem instances
SERVER_URL = 'https://cfds-ccm-prod.orm.datarobot.com/'

t1 = time.time()

# Using a project and model ID to score on the modeling server
# !/anaconda3/bin/batch_scoring --host=$SERVER_URL --user=$USERNAME --api_token=$API_TOKEN --datarobot_key=$DATAROBOT_KEY $PROJECT_ID $MODEL_ID $PRED_FILE --max_prediction_explanations=3 --out=out.csv

# Using a deployment to score on the prediction server
# !/anaconda3/bin/batch_scoring_deployment_aware --host=$SERVER_URL
#                                                --user=$USERNAME 
#                                                --api_token=$API_TOKEN 
#                                                --datarobot_key=$DATAROBOT_KEY
#                                                $DEPLOYMENT_ID 
#                                                $PRED_FILE 
#                                                --max_prediction_explanations=3 
#                                                --out=out.csv

# Make sure you point to your batch_scoring_deployment_aware location
sh_loc = !which batch_scoring_deployment_aware
sh_loc = sh_loc[0]
print(sh_loc + '\n')

# !~/opt/anaconda3/bin/batch_scoring_deployment_aware --host=$SERVER_URL --user=$USERNAME --api_token=$API_TOKEN --datarobot_key=$DATAROBOT_KEY $DEPLOYMENT_ID $PRED_FILE --max_prediction_explanations=3 --out=out.csv
!$sh_loc --host=$SERVER_URL --user=$USERNAME --api_token=$API_TOKEN --datarobot_key=$DATAROBOT_KEY $DEPLOYMENT_ID $PRED_FILE --max_prediction_explanations=3 --out=out.csv

print('- Time: %0.3f' % (time.time()-t1))

# copy over the batch_scorinng output to another file so you can run this cell again without a 'file exists' error
!cp out.csv out_keep.csv
!rm out.csv

# read in the csv_keep file to inspect it
df_batch_out = pd.read_csv('out_keep.csv')
df_batch_out.head()

/Users/matthew.cohen/opt/anaconda3/bin/batch_scoring_deployment_aware

MainProcess [INFO] version: 1.16.5
MainProcess [INFO] platform: darwin 3.7.5 (default, Oct 25 2019, 10:52:18) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
MainProcess [INFO] Will be using API endpoint: https://cfds-ccm-prod.orm.datarobot.com/predApi/v1.0/
MainProcess [INFO] auto_sampler: total time seconds - 0.009423017501831055
MainProcess [INFO] auto_sample: will use batches of 4837 rows
MainProcess [INFO] Reader go...
MainProcess [INFO] Writer go...
MainProcess [INFO] Network go...
Shovel_Proc [INFO] Shovel process started
Shovel_Proc [INFO] chunking 1000 rows took 0.020348072052001953
Shovel_Proc [INFO] shoveling complete | total time elapsed 0.022555828094482422s
Netwrk_Proc [INFO] 1 responses sent | time elapsed 0.05669403076171875s
MainProcess [INFO] shovel proc finished, exit code: 0
Writer_Proc [INFO] batch 0-1000 checkpointed
MainProcess [INFO] Writer progress: Results: 1 Written: 1 Rows done: 1000 User time: 0

,row_id,0.0,1.0,explanation_1_feature,explanation_1_strength,explanation_2_feature,explanation_2_strength,explanation_3_feature,explanation_3_strength
0,0,0.730244,0.269756,int_rate,0.559892,annual_inc,0.227116,earliest_cr_line (Year),-0.211149
1,1,0.740596,0.259404,int_rate,0.896423,loan_amnt,0.729536,annual_inc,-0.410705
2,2,0.965693,0.034307,int_rate,-0.777624,desc,-0.326922,purpose,-0.146544
3,3,0.920579,0.079421,int_rate,-0.744992,title,-0.395564,annual_inc,0.378500
4,4,0.670737,0.329263,int_rate,0.688230,annual_inc,0.353972,addr_state,0.247427


# 4. Check health status of a model, retrain a new model, and redeploy it

In [9]:
# 
# This checks the health of a depoloyed model, and if it is failing (iow, has drifted) then retrain it
#

# Let's time this:
t1 = time.time()

ENDPOINT = 'https://app.datarobot.com/api/v2'
# DEPLOYMENT_ID = ''  # declared above

headers = {'Content-Type': 'application/json', 'Authorization': 'token %s' % API_TOKEN}

print('Getting model deployments')
health_response = requests.get('%s/modelDeployments/%s/' % (ENDPOINT, DEPLOYMENT_ID), 
                               headers=headers)

if health_response.json()['modelHealth'] == 'failing':
    
    print('Getting a failing model')
    model = dr.Model.get(model_id=health_response.json()['model']['id'], 
                                project=health_response.json()['project']['id'])
    
    print('Starting a project to retrain the model in manual mode')
    retrainProject = dr.Project.start(sourcedata=TRAIN_FILE, 
                                      project_name='Lending Club Retrain', target='is_bad', 
                                      autopilot_on=False,
                                      worker_count=4)
    
    print('Retraining the model')
    modelJobId = retrainProject.train(model.blueprint)
    newModel = dr.models.modeljob.wait_for_async_model_creation(project_id=retrainProject.id, 
                                                                model_job_id=modelJobId)
    
    print('Updating the deployment to point to the new model')
    model_Update = requests.patch('%s/modelDeployments/%s/model' % (ENDPOINT, DEPLOYMENT_ID), 
                                  headers=headers, data="{'modelId':'%s'}" % newModel.id)
    
    pprint.pprint(model_Update)

print('- Time: %0.3f' % (time.time()-t1))

pprint.pprint(health_response.json())

Getting model deployments
- Time: 0.792
{'accuracyHealth': 'unavailable',
 'accuracyHealthEnd': None,
 'accuracyHealthMessage': 'Accuracy is unknown.',
 'accuracyHealthStart': None,
 'associationIdSettings': {'allowMissingValues': False, 'columnName': None},
 'createdAt': '2020-02-08 01:30:29.343000',
 'dataSourceTypes': ['scoring', 'training'],
 'deployed': False,
 'description': '',
 'externalDataInfo': None,
 'id': '5e3e0f3593165c12c2520a7f',
 'instance': {'datarobotKey': '544ec55f-61bf-f6ee-0caf-15c7f919a45d',
              'hostName': 'cfds-ccm-prod.orm.datarobot.com',
              'id': '5a61d7a0fbd723001a2f70d9',
              'sslEnabled': True},
 'label': 'is_bad Predictions',
 'lastPredictionRequest': '2020-02-10 00:35:08',
 'model': {'id': '5e3cc0accf4c157a1f5d6b5c',
           'isAnomalyDetection': False,
           'modelType': 'Nystroem Kernel SVM Classifier',
           'predictionThreshold': 0.5,
           'userId': '5a8a6402b11ba422e62b7c7a'},
 'modelHealth': 'unknow

## Dev

In [10]:
# Alt method to flatten pe's
# - https://stackoverflow.com/questions/25511765/pandas-expand-json-field-across-records
# import json
# def json_to_series(text):
#     text = str(text).replace("'", '"')
#     print(text)
#     keys, values = zip(*[(dct['feature'], dct['featureValue']) 
#                          for dct in json.loads(text)])
#     return pd.Series(values, index=keys)

# df = df_pe
# result = pd.concat([df, df['predictionExplanations'].apply(json_to_series)], axis=1)
# df['predictionExplanations'].apply(json_to_series)
# # df['predictionExplanations']
# # df.info()
# result

In [11]:
# batch script call
# print('%s --host=%s --user=%s --api_token=%s --datarobot_key=%s %s %s --max_prediction_explanations=3 --out=out.csv' % \
#     (sh_loc, SERVER_URL, USERNAME, API_TOKEN, DATAROBOT_KEY, DEPLOYMENT_ID, PRED_FILE))